In [1]:
 import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import os

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.under_sampling import ClusterCentroids
from sklearn.datasets import make_blobs
from imblearn.combine import SMOTEENN

In [4]:
# Load the data
file_path1 = Path('lending_data.csv')
df = pd.read_csv(file_path1)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [5]:
totaldf= pd.get_dummies(df, columns=["homeowner"])
totaldf.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,low_risk,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,low_risk,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,low_risk,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,low_risk,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,low_risk,1,0,0


In [6]:
# Create our features
X = totaldf.drop(columns='loan_status')

# Create our target
y = totaldf['loan_status']

In [7]:
X

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
0,10700.0,7.672,52800,0.431818,5,1,22800,0,1,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0,1,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0,0,1
3,10700.0,7.664,52700,0.430740,5,1,22700,0,1,0
4,10800.0,7.698,53000,0.433962,5,1,23000,1,0,0
...,...,...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600,0,1,0
77532,17700.0,10.662,80900,0.629172,11,2,50900,1,0,0
77533,17600.0,10.595,80300,0.626401,11,2,50300,0,0,1
77534,16300.0,10.068,75300,0.601594,10,2,45300,1,0,0


In [8]:
y.to_frame()

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk
...,...
77531,high_risk
77532,high_risk
77533,high_risk
77534,high_risk


In [9]:
X.describe() 

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [10]:
# Check the balance of our target values
y.value_counts(normalize=False)

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [11]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=1, stratify=y)

In [12]:
X_train.shape 

(58152, 10)

### Data Pre-Processing

In [13]:
data_scaler = StandardScaler()

In [14]:
scalex=data_scaler.fit(X_train)
scalex

StandardScaler()

In [15]:
x_scaled = scalex.transform(X_train)
x_scaledtest=scalex.transform(X_test)

In [16]:
x_scaledtest[:5]

array([[ 0.80958089,  0.81480327,  0.80977921,  1.06689092,  0.61664652,
         1.04495557,  0.80977921, -0.99273515,  1.22158265, -0.3377677 ],
       [-0.05072219, -0.07373693, -0.07456551,  0.06584169,  0.09115188,
        -0.67471435, -0.07456551,  1.00731801, -0.81861018, -0.3377677 ],
       [ 3.39049014,  3.39606878,  3.39110974,  2.89689483,  3.24411968,
         2.76462549,  3.39110974, -0.99273515,  1.22158265, -0.3377677 ],
       [ 0.37942935,  0.38178051,  0.37955746,  0.61521057,  0.61664652,
         1.04495557,  0.37955746,  1.00731801, -0.81861018, -0.3377677 ],
       [ 0.71399166,  0.72369978,  0.72612498,  0.98366993,  0.61664652,
         1.04495557,  0.72612498,  1.00731801, -0.81861018, -0.3377677 ]])

### Simple Logistic Regression

In [17]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [18]:
y_pred = model.predict(x_scaledtest) 

In [19]:
 # Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.8041461911615757

In [20]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [ 7258, 11501]])

In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      1.00      0.61      0.15      0.78      0.63       625
   low_risk       1.00      0.61      1.00      0.76      0.78      0.59     18759

avg / total       0.97      0.63      0.98      0.74      0.78      0.59     19384



### Oversampling

#### Naive Random Oversampling

In [22]:
randomos = RandomOverSampler(random_state=1)
X_resampled, y_resampled = randomos.fit_resample(x_scaled, y_train)
Counter(y_resampled)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [23]:
logreg = LogisticRegression(solver='lbfgs', random_state=1)
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [24]:
y_pred1 = logreg.predict(x_scaledtest) 
balanced_accuracy_score(y_test, y_pred1)

0.9946414201183431

In [25]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred1)

array([[  622,     3],
       [  111, 18648]])

In [26]:
print(classification_report_imbalanced(y_test, y_pred1))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [27]:
X_resampledso, y_resampledso = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    x_scaled, y_train)

Counter(y_resampledso)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [28]:
# Train the Logistic Regression model using the resampled data
modelsmote = LogisticRegression(solver='lbfgs', random_state=1)
modelsmote.fit(X_resampledso, y_resampledso)

LogisticRegression(random_state=1)

In [29]:
# Calculated the balanced accuracy score
y_predsmote = modelsmote.predict(x_scaledtest)
balanced_accuracy_score(y_test, y_predsmote)

0.9946680739911509

In [30]:
# Display the confusion matrix
confusion_matrix(y_test, y_predsmote)

array([[  622,     3],
       [  110, 18649]])

In [31]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predsmote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### Undersampling

In [32]:
cc = ClusterCentroids(random_state=1)

In [33]:
X_resampledcluster, y_resampledcluster = cc.fit_resample(x_scaled, y_train)

In [34]:
Counter(y_resampledcluster) 

Counter({'high_risk': 1875, 'low_risk': 1875})

In [35]:
# Train the Logistic Regression model using the resampled data
modelcc = LogisticRegression(solver='lbfgs', random_state=1)
modelcc.fit(X_resampledcluster, y_resampledcluster)

LogisticRegression(random_state=1)

In [36]:
y_predcluster=modelcc.predict(x_scaledtest)
balanced_accuracy_score(y_test, y_predcluster)

0.9932813049736127

In [37]:
# Display the confusion matrix
confusion_matrix(y_test, y_predcluster)

array([[  620,     5],
       [  102, 18657]])

In [38]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predcluster))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



### Combination (Over and Under) Sampling

In [39]:
sm = SMOTEENN(random_state=1)
X_resampledsm, y_resampledsm = sm.fit_resample(x_scaled, y_train)
Counter(y_resampledsm)

Counter({'high_risk': 55428, 'low_risk': 55906})

In [40]:
# Train the Logistic Regression model using the resampled data
modelsm = LogisticRegression(solver='lbfgs', random_state=1)
modelsm.fit(X_resampledsm, y_resampledsm)

LogisticRegression(random_state=1)

In [41]:
# Calculate the balanced accuracy score
y_predsm = modelsm.predict(x_scaledtest)
balanced_accuracy_score(y_test, y_predsm)

0.9946680739911509

In [42]:
confusion_matrix(y_test, y_predsm)

array([[  622,     3],
       [  110, 18649]])

In [43]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predsm))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384

